# TextAttack Imperceptible Perturbation Notebook
This notebook runs different adversarial attacks using TextAttack.
Select the perturbation type below.

In [2]:
# Configure experiment
perturbation_type = 'homoglyphs'  # options: 'homoglyphs', 'invisible', 'deletions', 'reorderings'
store_temp_files = False
store_results = False
class Args:
    pass
args = Args()
args.perturbation_type = perturbation_type
args.store_temp_files = store_temp_files
args.store_results = store_results

In [2]:
!pip install -r requirements.txt

In [1]:
import sys
import textattack
from textattack.models.wrappers import ModelWrapper
from typing import List, Tuple
from transformers import pipeline
from datasets import load_dataset
from string import punctuation
import argparse
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import os
import tarfile
import requests
from io import BytesIO
from bs4 import BeautifulSoup
import torch
import importlib.util
from pathlib import Path
import numpy as np
import pandas as pd
import shutil
import subprocess
import zipfile
import json
from collections import OrderedDict
from torch.nn.functional import softmax

In [6]:
class EmotionWrapper(ModelWrapper):

    def __init__(self, model):
        self.model = model

    def __call__(self, input_texts: List[str]) -> List[List[float]]:

        """
        Args:
            input_texts: List[str]

        Return:
            ret: List[List[float]]
            a list of elements, one per element of input_texts. Each element is a list of probabilities, one for each label.
        """
        ret = []
        for i in input_texts:
            pred = self.model(i)[0]
            scores = []
            for j in pred:
                scores.append(j['score'])
            ret.append(scores)
        return ret

model = pipeline("text-classification", model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True, device=-1)
model_wrapper = EmotionWrapper(model)

attack = textattack.attack_recipes.BadCharacters2021.build(
    model_wrapper, 
    goal_function_type="targeted_bonus",
    perturbation_type=args.perturbation_type
)
dataset = textattack.datasets.HuggingFaceDataset("emotion", split="test")
print(dataset[0])
attack_args = textattack.AttackArgs(
    num_examples=250,
    log_to_csv="results/emotion/log.csv"
)
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

if args.store_results == False:
    if os.path.isdir("results/emotion"):
        shutil.rmtree("results/emotion")


Device set to use cpu
/opt/anaconda3/envs/projenvconda39/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
textattack: No entry found for goal function <class 'textattack.goal_functions.custom.targeted_bonus.TargetedBonus'>.
textattack: Unknown if model of class <class 'transformers.pipelines.text_classification.TextClassificationPipeline'> compatible with goal function <class 'textattack.goal_functions.custom.targeted_bonus.TargetedBonus'>.
textattack: Loading datasets dataset emotion, split test.
textattack: Logging to CSV at path results/emotion/log.csv


(OrderedDict([('text', 'im feeling rather rotten so im not very ambitious right now')]), 0)
Attack(
  (search_method): DifferentialEvolution(
    (popsize):  32
    (maxiter):  10
    (max_perturbs):  1
    (verbose):  False
  )
  (goal_function):  TargetedBonus
  (transformation):  WordSwapHomoglyphSwap
  (constraints): None
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 1/250 [00:00<02:11,  1.89it/s]

tensor([9.9889e-01, 2.2335e-04, 2.2042e-04, 2.9064e-04, 1.7530e-04, 1.9613e-04]) --> tensor([9.9899e-01, 1.9190e-04, 1.8884e-04, 2.6047e-04, 1.9274e-04, 1.8098e-04])

im feeling rather rotten so im not very ambitious right now

im feeling rather rotten so im not very ambitіous right now
--------------------------------------------- Result 1 ---------------------------------------------
tensor([9.9889e-01, 2.2335e-04, 2.2042e-04, 2.9064e-04, 1.7530e-04, 1.9613e-04]) --> tensor([9.9899e-01, 1.9190e-04, 1.8884e-04, 2.6047e-04, 1.9274e-04, 1.8098e-04])

im feeling rather rotten so im not very [[ambitious]] right now

im feeling rather rotten so im not very [[ambitіous]] right now




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   1%|          | 2/250 [00:00<01:55,  2.15it/s]

...

tensor([3.4462e-04, 2.5880e-04, 2.2236e-04, 9.9782e-01, 1.1622e-03, 1.9423e-04]) --> [FAILED]

i feel like my irritable sensitive combination skin has finally met it s match
--------------------------------------------- Result 248 ---------------------------------------------
tensor([3.4462e-04, 2.5880e-04, 2.2236e-04, 9.9782e-01, 1.1622e-03, 1.9423e-04]) --> [[[FAILED]]]

i feel like my irritable sensitive combination skin has finally met it s match




[Succeeded / Failed / Skipped / Total] 111 / 138 / 0 / 249: 100%|█████████▉| 249/250 [03:08<00:00,  1.32it/s]

tensor([2.3530e-04, 9.9882e-01, 2.5060e-04, 1.7010e-04, 1.8717e-04, 3.3663e-04]) --> [FAILED]

i like it though its very over the top but makes me feel clever by association
--------------------------------------------- Result 249 ---------------------------------------------
tensor([2.3530e-04, 9.9882e-01, 2.5060e-04, 1.7010e-04, 1.8717e-04, 3.3663e-04]) --> [[[FAILED]]]

i like it though its very over the top but makes me feel clever by association




[Succeeded / Failed / Skipped / Total] 111 / 139 / 0 / 250: 100%|██████████| 250/250 [03:09<00:00,  1.32it/s]

tensor([0.0023, 0.0048, 0.0011, 0.0012, 0.3177, 0.6730]) --> [FAILED]

i will practice meditation if i feel overwhelmed and hopefully become successful in peaceful practice
--------------------------------------------- Result 250 ---------------------------------------------
tensor([0.0023, 0.0048, 0.0011, 0.0012, 0.3177, 0.6730]) --> [[[FAILED]]]

i will practice meditation if i feel overwhelmed and hopefully become successful in peaceful practice



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 111    |
| Number of failed attacks:     | 139    |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 55.6%  |
| Attack success rate:          | 44.4%  |
| Average perturbed word %:     | 7.57%  |
| Average num. words per input: | 18.75  |
| Avg num queries:              | 240.18 |
+---------------------------

In [5]:
class NERModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model

    def __call__(self, input_texts: List[str]):
        """
        Args:
            input_texts: List[str]
        
        Return:
            ret
                Model output
        """
        ret = []
        for i in input_texts:
            pred = self.model(i)
            ret.append(pred)
        return ret

model = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")
model_wrapper = NERModelWrapper(model)
ner_classes = ['PER', 'ORG', 'LOC', 'MISC']
attack = textattack.attack_recipes.BadCharacters2021.build(
    model_wrapper, 
    goal_function_type="named_entity_recognition", 
    perturbation_type=args.perturbation_type, 
    target_suffix=ner_classes[0]
)
dataset = load_dataset("conll2003", split="test", trust_remote_code=True)
pairs = []
def detokenize(tokens: List[str]) -> str:
    output = ""
    for index, token in enumerate(tokens):
        if (len(token) == 1 and token in punctuation) or index == 0:
            output += token
        else:
            output += ' ' + token
    return output
for ex in dataset:
    tokens = ex["tokens"]
    ner_labels = ex["ner_tags"]
    text = detokenize(tokens) 
    pairs.append((text, "NER")) # hack
dataset = textattack.datasets.Dataset(pairs)
attack_args = textattack.AttackArgs(
    num_examples=10,
    log_to_csv="results/named_entity_recognition/log.csv"
)
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()
if args.store_results == False:
    if os.path.isdir("results/named_entity_recognition"):
        shutil.rmtree("results/named_entity_recognition")

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
textattack: No entry found for goal function <class 'textattack.goal_functions.custom.named_entity_recognition.NamedEntityRecognition'>.
textattack: Unknown if mod

Attack(
  (search_method): DifferentialEvolution(
    (popsize):  32
    (maxiter):  10
    (max_perturbs):  1
    (verbose):  False
  )
  (goal_function):  NamedEntityRecognition
  (transformation):  WordSwapHomoglyphSwap
  (constraints): None
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:04<00:44,  4.99s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[
  {
    "entity": "I-MISC",
    "score": 0.9950999617576599,
    "index": 6,
    "word": "J",
    "start": 8,
    "end": 9
  },
  {
    "entity": "I-MISC",
    "score": 0.9684495329856873,
    "index": 7,
    "word": "##AP",
    "start": 9,
    "end": 11
  },
  {
    "entity": "I-MISC",
    "score": 0.9932603240013123,
    "index": 8,
    "word": "##AN",
    "start": 11,
    "end": 13
  },
  {
    "entity": "I-LOC",
    "score": 0.541292130947113,
    "index": 18,
    "word": "CH",
    "start": 29,
    "end": 31
  },
  {
    "entity": "I-LOC",
    "score": 0.5053967237472534,
    "index": 19,
    "word": "##IN",
    "start": 31,
    "end": 33
  },
  {
    "entity": "I-ORG",
    "score": 0.7646154761314392,
    "index": 20,
    "word": "##A",
    "start": 33,
    "end": 34
  }
] --> [
  {
    "entity": "I-MISC",
    "score": 0.9942578077316284,
    "index": 6,
    "word": "J",
    "sta

[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  20%|██        | 2/10 [00:05<00:23,  2.91s/it]

...

--------------------------------------------- Result 8 ---------------------------------------------
[
  {
    "entity": "I-MISC",
    "score": 0.9985392093658447,
    "index": 3,
    "word": "Soviet",
    "start": 11,
    "end": 17
  },
  {
    "entity": "I-MISC",
    "score": 0.9799859523773193,
    "index": 9,
    "word": "Asian",
    "start": 45,
    "end": 50
  },
  {
    "entity": "I-MISC",
    "score": 0.9974244832992554,
    "index": 10,
    "word": "Cup",
    "start": 51,
    "end": 54
  }
] --> [[[FAILED]]]

The former Soviet republic was playing in an Asian Cup finals tie for the first time.




[Succeeded / Failed / Skipped / Total] 5 / 4 / 0 / 9:  90%|█████████ | 9/10 [00:32<00:03,  3.64s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[
  {
    "entity": "I-MISC",
    "score": 0.9735670685768127,
    "index": 4,
    "word": "Asian",
    "start": 20,
    "end": 25
  },
  {
    "entity": "I-MISC",
    "score": 0.9971873164176941,
    "index": 5,
    "word": "Games",
    "start": 26,
    "end": 31
  },
  {
    "entity": "I-LOC",
    "score": 0.9997908473014832,
    "index": 11,
    "word": "Uzbekistan",
    "start": 53,
    "end": 63
  }
] --> [
  {
    "entity": "I-PER",
    "score": 0.637923002243042,
    "index": 1,
    "word": "D",
    "start": 0,
    "end": 1
  },
  {
    "entity": "I-ORG",
    "score": 0.7454715967178345,
    "index": 2,
    "word": "##е",
    "start": 1,
    "end": 2
  },
  {
    "entity": "I-ORG",
    "score": 0.8040305972099304,
    "index": 3,
    "word": "##sp",
    "start": 2,
    "end": 4
  },
  {
    "entity": "I-ORG",
    "score": 0.8762766122817993,
    "index": 4,
    "word": "##ite",
 

[Succeeded / Failed / Skipped / Total] 5 / 5 / 0 / 10: 100%|██████████| 10/10 [00:36<00:00,  3.63s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[
  {
    "entity": "I-LOC",
    "score": 0.9997974038124084,
    "index": 12,
    "word": "Japan",
    "start": 64,
    "end": 69
  },
  {
    "entity": "I-LOC",
    "score": 0.9998229146003723,
    "index": 27,
    "word": "Syria",
    "start": 154,
    "end": 159
  }
] --> [[[FAILED]]]

Two goals from defensive errors in the last six minutes allowed Japan to come from behind and collect all three points from their opening meeting against Syria.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 5      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 50.0%  |
| Attack success rate:          | 50.0%  |
| Average perturbed word %:     | 14.91% |
| Average num

In [ ]:
class FairseqTranslationWrapper(ModelWrapper):
    """
    A wrapper for the model
        torch.hub.load('pytorch/fairseq',
                        'transformer.wmt14.en-fr',
                        tokenizer='moses',
                        bpe='subword_nmt',
                        verbose=False).eval()
    or any other model with a .translate() method.
    """

    def __init__(self, model):
        self.model = model  

    def __call__(self, text_input_list: List[str]) -> List[str]:
        """
        Args:
            input_texts: List[str]
        
        Return:
            ret: List[str]
                Result of translation. One per element in input_texts.
        """
        return [self.model.translate(text) for text in text_input_list]
model = torch.hub.load(
    'pytorch/fairseq',
    'transformer.wmt14.en-fr',
    tokenizer='moses',
    bpe='subword_nmt',
    verbose=False
).eval()

model_wrapper = FairseqTranslationWrapper(model)
def download_en_fr_dataset():
    

    # Define constants
    url = "http://statmt.org/wmt14/test-full.tgz"
    target_dir = os.path.join("temp/translation", "data")
    os.makedirs(target_dir, exist_ok=True)

    print(f"Downloading WMT14 test data from {url}...")

    # Download and extract in-memory
    response = requests.get(url, stream=True)
    response.raise_for_status()

    with tarfile.open(fileobj=BytesIO(response.content), mode="r:gz") as tar:
        for member in tar.getmembers():
            if member.name.endswith("newstest2014-fren-src.en.sgm") or member.name.endswith("newstest2014-fren-ref.fr.sgm"):
                print(f"Extracting {member.name} to {target_dir}")
                member.name = os.path.basename(member.name) 
                tar.extract(member, path=target_dir)

    print("en_fr dataset downloaded.")
def load_en_fr_dataset():
    """
    Loads English-French sentence pairs from SGM files and returns a TextAttack Dataset.

    Returns:
        textattack.datasets.Dataset: wrapped dataset of (English, French) pairs.
    """
    

    source_path = os.path.join("temp/translation", "data/newstest2014-fren-src.en.sgm")
    target_path = os.path.join("temp/translation", "data/newstest2014-fren-ref.fr.sgm")

    with open(source_path, "r", encoding="utf-8") as f:
        source_doc = BeautifulSoup(f, "html.parser")

    with open(target_path, "r", encoding="utf-8") as f:
        target_doc = BeautifulSoup(f, "html.parser")

    pairs = []

    for doc in source_doc.find_all("doc"):
        docid = str(doc["docid"])
        for seg in doc.find_all("seg"):
            segid = str(seg["id"])
            src = str(seg.string).strip() if seg.string else ""
            tgt_node = target_doc.select_one(f'doc[docid="{docid}"] > seg[id="{segid}"]')
            if tgt_node and tgt_node.string:
                tgt = str(tgt_node.string).strip()
                pairs.append((src, tgt))
    return textattack.datasets.Dataset(pairs) 
download_en_fr_dataset()
dataset = load_en_fr_dataset()
attack = textattack.attack_recipes.BadCharacters2021.build(
    model_wrapper, 
    goal_function_type="maximize_levenshtein", 
    perturbation_type=args.perturbation_type,
    target_distance=0.1
)
print(dataset[0])
attack_args = textattack.AttackArgs(
    num_examples=1,
    log_to_csv="results/translation/log.csv"
)
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

# if args.store_temp_files == False:
#     if os.path.isdir("temp/translation"):
#         shutil.rmtree("temp/translation")
# if args.store_results == False:
#     if os.path.isdir("results/translation"):
#         shutil.rmtree("results/translation")

/Users/vlwk/.cache/torch/hub/pytorch_fairseq_main/fairseq/checkpoint_utils.py:340: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f, map_location=torch.dev

Extracting test-full/newstest2014-fren-ref.fr.sgm to temp/translation/data
Extracting test-full/newstest2014-fren-src.en.sgm to temp/translation/data
en_fr dataset downloaded.


textattack: No entry found for goal function <class 'textattack.goal_functions.text.maximize_levenshtein.MaximizeLevenshtein'>.
textattack: Unknown if model of class <class 'fairseq.hub_utils.GeneratorHubInterface'> compatible with goal function <class 'textattack.goal_functions.text.maximize_levenshtein.MaximizeLevenshtein'>.
textattack: Logging to CSV at path results/translation/log.csv


(OrderedDict([('text', 'Spectacular Wingsuit Jump Over Bogota')]), 'Spectaculaire saut en "wingsuit" au-dessus de Bogota')
Attack(
  (search_method): DifferentialEvolution(
    (popsize):  32
    (maxiter):  10
    (max_perturbs):  1
    (verbose):  False
  )
  (goal_function):  MaximizeLevenshtein(
    (maximizable):  False
    (target_distance):  0.1
  )
  (transformation):  WordSwapHomoglyphSwap
  (constraints): None
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1: 100%|██████████| 1/1 [00:11<00:00, 11.71s/it]

--------------------------------------------- Result 1 ---------------------------------------------
Spectaculaire combinaison pour les ailes sauter au-dessus de Bogota --> Spectaculaire saut Wingѕuit au-dessus de Bogota

Spectacular [[Wingsuit]] Jump Over Bogota

Spectacular [[Wingѕuit]] Jump Over Bogota



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 20.0%  |
| Average num. words per input: | 5.0    |
| Avg num queries:              | 291.0  |
+-------------------------------+--------+


In [5]:
class FairseqTranslationWrapper(ModelWrapper):
    """
    A wrapper for the model
        torch.hub.load('pytorch/fairseq',
                        'transformer.wmt14.en-fr',
                        tokenizer='moses',
                        bpe='subword_nmt',
                        verbose=False).eval()
    or any other model with a .translate() method.
    """

    def __init__(self, model):
        self.model = model  

    def __call__(self, text_input_list: List[str]) -> List[str]:
        """
        Args:
            input_texts: List[str]
        
        Return:
            ret: List[str]
                Result of translation. One per element in input_texts.
        """
        return [self.model.translate(text) for text in text_input_list]
model = torch.hub.load(
    'pytorch/fairseq',
    'transformer.wmt14.en-fr',
    tokenizer='moses',
    bpe='subword_nmt',
    verbose=False
).eval()

model_wrapper = FairseqTranslationWrapper(model)
def download_en_fr_dataset():
    

    # Define constants
    url = "http://statmt.org/wmt14/test-full.tgz"
    target_dir = os.path.join("temp/translation", "data")
    os.makedirs(target_dir, exist_ok=True)

    print(f"Downloading WMT14 test data from {url}...")

    # Download and extract in-memory
    response = requests.get(url, stream=True)
    response.raise_for_status()

    with tarfile.open(fileobj=BytesIO(response.content), mode="r:gz") as tar:
        for member in tar.getmembers():
            if member.name.endswith("newstest2014-fren-src.en.sgm") or member.name.endswith("newstest2014-fren-ref.fr.sgm"):
                print(f"Extracting {member.name} to {target_dir}")
                member.name = os.path.basename(member.name) 
                tar.extract(member, path=target_dir)

    print("en_fr dataset downloaded.")
def load_en_fr_dataset():
    """
    Loads English-French sentence pairs from SGM files and returns a TextAttack Dataset.

    Returns:
        textattack.datasets.Dataset: wrapped dataset of (English, French) pairs.
    """
    

    source_path = os.path.join("temp/translation", "data/newstest2014-fren-src.en.sgm")
    target_path = os.path.join("temp/translation", "data/newstest2014-fren-ref.fr.sgm")

    with open(source_path, "r", encoding="utf-8") as f:
        source_doc = BeautifulSoup(f, "html.parser")

    with open(target_path, "r", encoding="utf-8") as f:
        target_doc = BeautifulSoup(f, "html.parser")

    pairs = []

    for doc in source_doc.find_all("doc"):
        docid = str(doc["docid"])
        for seg in doc.find_all("seg"):
            segid = str(seg["id"])
            src = str(seg.string).strip() if seg.string else ""
            tgt_node = target_doc.select_one(f'doc[docid="{docid}"] > seg[id="{segid}"]')
            if tgt_node and tgt_node.string:
                tgt = str(tgt_node.string).strip()
                pairs.append((src, tgt))
    return textattack.datasets.Dataset(pairs) 
download_en_fr_dataset()
dataset = load_en_fr_dataset()
attack = textattack.attack_recipes.BadCharacters2021.build(
    model_wrapper, 
    goal_function_type="maximize_levenshtein", 
    perturbation_type="homoglyphs",
    popsize=5,
    maxiter=3,
    perturbs=5
)
print(dataset[0])
attack_args = textattack.AttackArgs(
    num_examples=50,
    log_to_csv="results/translation/fairseq/pert3/log.csv",
    checkpoint_interval=5,
    checkpoint_dir=f"results/translation/fairseq/pert3",
)
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

# if args.store_temp_files == False:
#     if os.path.isdir("temp/translation"):
#         shutil.rmtree("temp/translation")
# if args.store_results == False:
#     if os.path.isdir("results/translation"):
#         shutil.rmtree("results/translation")

/Users/vlwk/.cache/torch/hub/pytorch_fairseq_main/fairseq/checkpoint_utils.py:340: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f, map_location=torch.dev

Extracting test-full/newstest2014-fren-ref.fr.sgm to temp/translation/data
Extracting test-full/newstest2014-fren-src.en.sgm to temp/translation/data
en_fr dataset downloaded.


textattack: No entry found for goal function <class 'textattack.goal_functions.text.maximize_levenshtein.MaximizeLevenshtein'>.
textattack: Unknown if model of class <class 'fairseq.hub_utils.GeneratorHubInterface'> compatible with goal function <class 'textattack.goal_functions.text.maximize_levenshtein.MaximizeLevenshtein'>.
textattack: Logging to CSV at path results/translation/fairseq/pert3/log.csv


(OrderedDict([('text', 'Spectacular Wingsuit Jump Over Bogota')]), 'Spectaculaire saut en "wingsuit" au-dessus de Bogota')
Attack(
  (search_method): DifferentialEvolution(
    (popsize):  5
    (maxiter):  3
    (max_perturbs):  5
    (verbose):  False
  )
  (goal_function):  MaximizeLevenshtein(
    (maximizable):  False
    (target_distance):  None
  )
  (transformation):  WordSwapHomoglyphSwap
  (constraints): None
  (is_black_box):  True
) 



Spectaculaire combinaison pour les ailes sauter au-dessus de Bogota --> [FAILED]

Spectacular Wingsuit Jump Over Bogota
--------------------------------------------- Result 1 ---------------------------------------------
Spectaculaire combinaison pour les ailes sauter au-dessus de Bogota --> [[[FAILED]]]

Spectacular Wingsuit Jump Over Bogota




...

textattack: Saving checkpoint under "results/translation/fairseq/pert3/1746378643815.ta.chkpt" at 2025-05-04 18:10:43 after 50 attacks.


Si vous pouvez le faire, dit Khan, le public se sentira plus à l'aise. --> [FAILED]

If you can do that, Khan said, the public gets more comfortable.
--------------------------------------------- Result 50 ---------------------------------------------
Si vous pouvez le faire, dit Khan, le public se sentira plus à l'aise. --> [[[FAILED]]]

If you can do that, Khan said, the public gets more comfortable.







[Succeeded / Failed / Skipped / Total] 0 / 50 / 0 / 50: 100%|██████████| 50/50 [1:55:51<00:00, 139.03s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 50     |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 100.0% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 18.38  |
| Avg num queries:              | 107.0  |
+-------------------------------+--------+



/Users/vlwk/Documents/GitHub/TextAttack/textattack/metrics/attack_metrics/words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
/opt/anaconda3/envs/projenvconda39/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [7]:
class IBMMAXToxicWrapper(ModelWrapper):
    """
    A wrapper for the IBM Max Toxic model
    https://github.com/IBM/MAX-Toxic-Comment-Classifier/blob/master/core/model.py
    """
    def __init__(self, ibm_model_wrapper):
        """
        Args:
            ibm_model_wrapper: An instance of the IBM MAX Toxic `ModelWrapper()` class.
        """
        self.model = ibm_model_wrapper

    def __call__(self, input_text_list: List[str]) -> np.ndarray:
        """
        Args:
            input_texts: List[str]
        
        Return:
            ret: np.ndarray
                One entry per element in input_text_list. Each is a list of logits, one for each label.
        """
        self.model._pre_process(input_text_list)
        logits = self.model._predict(input_text_list)
        return np.array(logits)

# Download model

def run(cmd):
    print(f"Running: {cmd}")
    subprocess.run(cmd, shell=True, check=True)
os.makedirs("temp/toxic", exist_ok=True)
shutil.rmtree(os.path.join("temp/toxic", "assets"), ignore_errors=True)
shutil.rmtree(os.path.join("temp/toxic", "toxic"), ignore_errors=True)
os.makedirs(os.path.join("temp/toxic", "assets"), exist_ok=True)
url = "https://codait-cos-max.s3.us.cloud-object-storage.appdomain.cloud/max-toxic-comment-classifier/1.0.0/assets.tar.gz"
tar_path = os.path.join("temp/toxic", "assets/assets.tar.gz")
print(f"Downloading {url}")
response = requests.get(url)
with open(tar_path, "wb") as f:
    f.write(response.content)
print("Extracting assets...")
with tarfile.open(tar_path, "r:gz") as tar:
    tar.extractall("temp/toxic/assets")
os.remove(tar_path)
run("git clone https://github.com/IBM/MAX-Toxic-Comment-Classifier.git")
shutil.move("MAX-Toxic-Comment-Classifier", os.path.join("temp/toxic", "toxic"))
requirements_path = os.path.join("temp/toxic", "toxic/requirements.txt")
with open(requirements_path, "r") as f:
    lines = f.readlines()
with open(requirements_path, "w") as f:
    for line in lines:
        f.write(line.split("==")[0].strip() + "\n")
run(f"pip install -r {requirements_path}")
run("pip install maxfw")
model_py_path = os.path.join("temp/toxic", "toxic/core/model.py")
with open(model_py_path, "r") as f:
    content = f.read()
content = content.replace("from config", "from ..config")
content = content.replace("from core.", "from .")
with open(model_py_path, "w") as f:
    f.write(content)
with open("temp/toxic/toxic/config.py", "r") as f:
    content = f.read()
content = content.replace("assets", "temp/toxic/assets")
with open("temp/toxic/toxic/config.py", "w") as f:
    f.write(content)
importlib.invalidate_caches()

def load_model_wrapper():
    module_name = "temp.toxic.toxic.core.model"
    
    # Remove the module if it's already loaded
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])
    else:
        importlib.import_module(module_name)
    
    module = sys.modules[module_name]
    return module.ModelWrapper()

model = load_model_wrapper()
model_wrapper = IBMMAXToxicWrapper(model)

# Get data
if (os.path.exists("temp/toxic/toxicity_annotated_comments.tsv") == False) or (os.path.exists("temp/toxic/toxicity_annotations.tsv") == False): 
    data_urls = {
        "temp/toxic/toxicity_annotated_comments.tsv": "https://ndownloader.figshare.com/files/7394542",
        "temp/toxic/toxicity_annotations.tsv": "https://ndownloader.figshare.com/files/7394539",
    }
    for filename, url in data_urls.items():
        print(f"Downloading {filename}")
        response = requests.get(url)
        with open(filename, "wb") as f:
            f.write(response.content)

comments = pd.read_csv('temp/toxic/toxicity_annotated_comments.tsv', sep = '\t', index_col = 0)
annotations = pd.read_csv('temp/toxic/toxicity_annotations.tsv',  sep = '\t')
# labels a comment as toxic if the majority of annoatators did so
labels = annotations.groupby('rev_id')['toxicity'].mean() > 0.5
# join labels and comments
comments['toxicity'] = labels
# remove newline and tab tokens
comments['comment'] = comments['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
test_comments = comments.query("split=='test'").query("toxicity==True")
examples = test_comments.reset_index().to_dict('records')

pairs = [(row["comment"], 0) for row in examples]

dataset = textattack.datasets.Dataset(pairs)
print(dataset[0])
attack = textattack.attack_recipes.BadCharacters2021.build(
    model_wrapper, 
    goal_function_type="logit_sum", 
    perturbation_type=args.perturbation_type
)
attack_args = textattack.AttackArgs(
    num_examples=10,
    log_to_csv="results/toxic/log.csv"
)
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()
if args.store_temp_files == False:
    if os.path.isdir("temp/toxic"):
        shutil.rmtree("temp/toxic")
if args.store_results == False:
    if os.path.isdir("results/toxic"):
        shutil.rmtree("results/toxic")

Extracting assets...
Running: git clone https://github.com/IBM/MAX-Toxic-Comment-Classifier.git


Cloning into 'MAX-Toxic-Comment-Classifier'...


Running: pip install -r temp/toxic/toxic/requirements.txt
Running: pip install maxfw


/Users/vlwk/Documents/GitHub/TextAttack/tests/badcharacters2021/temp/toxic/toxic/core/model.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.loa

textattack: No entry found for goal function <class 'textattack.goal_functions.custom.logit_sum.LogitSum'>.
textattack: Unknown if model of class <class 'temp.toxic.toxic.core.model.ModelWrapper'> compatible with goal function <class 'textattack.goal_functions.custom.logit_sum.LogitSum'>.


...

--------------------------------------------- Result 8 ---------------------------------------------
tensor([0.9889, 0.0698, 0.9102, 0.0015, 0.9791, 0.0200]) --> [[[FAILED]]]

  == whoa ==  you are a bi fat idiot, stop spamming my user space.




[Succeeded / Failed / Skipped / Total] 3 / 6 / 0 / 9:  90%|█████████ | 9/10 [00:41<00:04,  4.62s/it]

--------------------------------------------- Result 9 ---------------------------------------------
tensor([0.9972, 0.5847, 0.9971, 0.0080, 0.9786, 0.0395]) --> [[[FAILED]]]

  == FUCK YOU THUE ==  YOU BRAIN DEAD FUCK, YOU CAN'T BLOCK ME. THANKS TO PROXY SERVERS AND ANON BROWSING I CAN COME TO WIKI ZILLION TIMES SO IF SCUM LIKE YOU THINK THAT YOU CAN BLOCK ME THEN YOU ARE IN FOR BIG SURPRISE YOU FAGGOT. I AM GIVING YOU LAST CHANCE IN CIVILIZED WAY TO STOP VANDALIZING KASH JAFFREY. DON'T FUCK WITH CANADIAN CONTENT, ITS TO ADVANCE FOR YOUR EMPTY BRAIN TO CONCIEVE. AND LASTLY IF YOU WANT TO LEAVE MESSAGE THEN WRITE (DON'T USE PREDEFINE ONE LINERS.) SEE YA ASSHOLE  JOEY




[Succeeded / Failed / Skipped / Total] 4 / 6 / 0 / 10: 100%|██████████| 10/10 [00:48<00:00,  4.81s/it]

--------------------------------------------- Result 10 ---------------------------------------------
tensor([7.6723e-01, 1.3083e-03, 2.5661e-01, 8.6072e-04, 2.6488e-03, 3.2799e-04]) --> tensor([4.4767e-04, 8.7316e-05, 1.3527e-04, 7.5941e-05, 1.3710e-04, 1.1336e-04])

`  :: Hey, the only persons ``suffering`` were those who read that [[bullshit]], Wikipedia opening itself up to a Copyright infringement suit, and the credibility of those who are facilitating both.  The length of the protections will not influence my decision to revert the article back to a previous version that does not contain plagiarism.  `

`  :: Hey, the only persons ``suffering`` were those who read that [[bullѕhit]], Wikipedia opening itself up to a Copyright infringement suit, and the credibility of those who are facilitating both.  The length of the protections will not influence my decision to revert the article back to a previous version that does not contain plagiarism.  `



+-------------------------------+

In [8]:
class FairseqMnliWrapper(ModelWrapper):
    """
    A wrapper for the model
        torch.hub.load('pytorch/fairseq', 'roberta.large.mnli').eval()
    """

    def __init__(self, model):
        self.model = model

    def __call__(self, input_texts: List[Tuple[str, str]]) -> List[torch.Tensor]:
        """
        Args:
            input_texts: List[Tuple[str, str]]
                List of (premise, hypothesis)
        
        Return:
            ret: List[torch.Tensor]
                Each tensor is a list of probabilities, 
                one for each of (contradiction, neutral, entailment)
        """
        ret = []
        for t in input_texts:
            premise = t[0]
            hypothesis = t[1]
            tokens = self.model.encode(premise, hypothesis)
            predict = self.model.predict('mnli', tokens)
            probs = softmax(predict, dim=1).cpu().detach()[0]
            ret.append(probs.unsqueeze(0))
        return ret
model = torch.hub.load('pytorch/fairseq',
    'roberta.large.mnli').eval()
model_wrapper = FairseqMnliWrapper(model)
url = "https://cims.nyu.edu/~sbowman/multinli/multinli_1.0.zip"
os.makedirs("temp/mnli", exist_ok=True)  
response = requests.get(url)
response.raise_for_status() 
with zipfile.ZipFile(BytesIO(response.content)) as zip_ref:
    zip_ref.extractall("temp/mnli")
label_map = {'contradiction': 0, 'neutral': 1, 'entailment': 2}
pairs = []
with open("temp/mnli/multinli_1.0/multinli_1.0_dev_matched.jsonl", 'r') as f:
    for line in f:
        sample = json.loads(line)
        if sample['gold_label'] not in label_map:
            continue

        premise = sample['sentence1']
        hypothesis = sample['sentence2']
        label = label_map[sample['gold_label']]
        item = ((premise, hypothesis), label)
        pairs.append(item)
dataset = textattack.datasets.Dataset(pairs, input_columns=["premise", "hypothesis"])
print(dataset[0])
attack = textattack.attack_recipes.BadCharacters2021.build(
    model_wrapper, 
    goal_function_type="targeted_strict", 
    perturbation_type=args.perturbation_type
)
attack_args = textattack.AttackArgs(
    num_examples=10,
    log_to_csv="results/mnli/log.csv"
)
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()
if args.store_temp_files == False:
    if os.path.isdir("temp/mnli"):
        shutil.rmtree("temp/mnli")
if args.store_results == False:
    if os.path.isdir("results/mnli"):
        shutil.rmtree("results/mnli")

Using cache found in /Users/vlwk/.cache/torch/hub/pytorch_fairseq_main
/Users/vlwk/.cache/torch/hub/pytorch_fairseq_main/fairseq/checkpoint_utils.py:340: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to th

(OrderedDict([('premise', 'The new rights are nice enough'), ('hypothesis', 'Everyone really likes the newest benefits ')]), 1)


textattack: Logging to CSV at path results/mnli/log.csv


Attack(
  (search_method): DifferentialEvolution(
    (popsize):  32
    (maxiter):  10
    (max_perturbs):  1
    (verbose):  False
  )
  (goal_function):  TargetedStrict
  (transformation):  WordSwapHomoglyphSwap
  (constraints): None
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:  10%|█         | 1/10 [00:03<00:30,  3.34s/it]

...

--------------------------------------------- Result 8 ---------------------------------------------
tensor([0.0014, 0.5608, 0.4378]) --> tensor([0.6773, 0.1157, 0.2070])

[[[[Premise]]]]: Calcutta seems to be the [[only]] other production center having any pretensions to artistic creativity at all, but ironically you're actually more likely to see the works of Satyajit Ray or Mrinal Sen shown in Europe or North America than in India itself.
[[[[Hypothesis]]]]: Most of Mrinal Sen's work can be found in European collections.

Calcutta seems to be the [[onlу]] other production center having any pretensions to artistic creativity at all, but ironically you're actually more likely to see the works of Satyajit Ray or Mrinal Sen shown in Europe or North America than in India itself.
Most of Mrinal Sen's work can be found in European collections.




[Succeeded / Failed / Skipped / Total] 4 / 5 / 0 / 9:  90%|█████████ | 9/10 [00:37<00:04,  4.20s/it]

--------------------------------------------- Result 9 ---------------------------------------------
tensor([0.8050, 0.1832, 0.0118]) --> tensor([0.6584, 0.1275, 0.2141])

[[[[Premise]]]]: If that investor were willing to pay extra for the security of limited downside, she could buy put options with a strike price of $98, which would lock [[in]] her profit on the shares at $18, less whatever the options cost.
[[[[Hypothesis]]]]: THe strike price could be $8.

If that investor were willing to pay extra for the security of limited downside, she could buy put options with a strike price of $98, which would lock [[іn]] her profit on the shares at $18, less whatever the options cost.
THe strike price could be $8.




[Succeeded / Failed / Skipped / Total] 4 / 6 / 0 / 10: 100%|██████████| 10/10 [00:42<00:00,  4.26s/it]

--------------------------------------------- Result 10 ---------------------------------------------
tensor([0.0309, 0.9665, 0.0026]) --> [[[FAILED]]]

[[[[Premise]]]]: 3)  Dare you rise to the occasion, like Raskolnikov, and reject the petty rules that govern lesser men?
[[[[Hypothesis]]]]: Would you rise up and defeaat all evil lords in the town?



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 6      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 60.0%  |
| Attack success rate:          | 40.0%  |
| Average perturbed word %:     | 2.11%  |
| Average num. words per input: | 34.7   |
| Avg num queries:              | 99.0   |
+-------------------------------+--------+

In [9]:
if args.store_temp_files == False:
    if os.path.isdir("temp"):
        shutil.rmtree("temp")

if args.store_results == False:
    if os.path.isdir("results"):
        shutil.rmtree("results")